# Comments
Thanks to tito and ragnar for this kernels https://www.kaggle.com/its7171/lgbm-with-loop-feature-engineering
https://www.kaggle.com/ragnar123/riiid-model-lgbm


In [ ]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb
#import riiideducation
import matplotlib.pyplot as plt
import seaborn as sns

import random
import os

In [ ]:
%%time

# Random seed
SEED = 123

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

qs = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
qs['tag'] = qs['tags'].apply(lambda ts: [int(x) for x in str(ts).split() if x != 'nan'])
qs_tags = qs[['question_id', 'tag']].set_index('question_id').to_dict()['tag']
qs_parts = qs[['question_id', 'part']].set_index('question_id').to_dict()['part']
del qs
gc.collect()


# Funcion for user stats with loops
def add_features(df, len_df, answered_correctly_u_count, answered_correctly_u_sum, elapsed_time_u_sum, explanation_u_sum, timestamp_u, timestamp_u_incorrect, answered_correctly_q_count, answered_correctly_q_sum, elapsed_time_q_sum, explanation_q_sum, answered_correctly_uq, last_wrong_container_u, last_container_u, previous_qu, previous_qa, answered_correctly_pu_count, answered_correctly_pu_sum, last20_answers_u, last20_uq_diff, container_uq_diff, diff_u_sum, diff_u_cnt, diff_pu_sum, diff_pu_cnt, row_from_lecture, update = True):
    # -----------------------------------------------------------------------
    # Client features
    #len_df = len(df)
    print(len_df)
    answered_correctly_u_avg = np.zeros(len_df, dtype = np.float32)
    answered_correctly_pu_avg = np.zeros(len_df, dtype = np.float32)
    answered_correctly_u_avg20 = np.zeros(len_df, dtype = np.float32)
    difficulty_u_avg20 = np.zeros(len_df, dtype = np.float32)
    difficulty_u_avg = np.zeros(len_df, dtype = np.float32)
    difficulty_pu_avg = np.zeros(len_df, dtype = np.float32)
    row_from_lecture_u = np.zeros(len_df, dtype = np.int32)
    answered_u_cnt = np.zeros(len_df, dtype = np.float32)
    answered_pu_cnt = np.zeros(len_df, dtype = np.float32)
    elapsed_time_u_avg = np.zeros(len_df, dtype = np.float32)
    explanation_u_avg = np.zeros(len_df, dtype = np.float32)
    timestamp_u_recency_1 = np.zeros(len_df, dtype = np.float32)
    timestamp_u_recency_2 = np.zeros(len_df, dtype = np.float32)
    timestamp_u_recency_3 = np.zeros(len_df, dtype = np.float32)
    timestamp_u_incorrect_recency = np.zeros(len_df, dtype = np.float32)
    # -----------------------------------------------------------------------
    # Question features
    answered_correctly_q_avg = np.zeros(len_df, dtype = np.float32)
    elapsed_time_q_avg = np.zeros(len_df, dtype = np.float32)
    explanation_q_avg = np.zeros(len_df, dtype = np.float32)
    #------------------------------------------------------------------
    # Container features
    wrong_container_u = np.zeros(len_df, dtype = np.int8)
    # -----------------------------------------------------------------------
    # User Question
    answered_correctly_uq_count = np.zeros(len_df, dtype = np.int32)
    # -----------------------------------------------------------------------
    num = -1
    
    for row in df[['user_id', 'answered_correctly', 'content_id', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'timestamp', 'task_container_id', 'tag', 'part']].values:
        
        #New question or lecture
        if row[1] != -1:
            num += 1
            row_from_lecture[row[0]] += 1
        else:
            row_from_lecture[row[0]] = 1
            continue
        
        # Previous container update if new container
        if last_container_u[row[0]] != row[6]:
            # Client features updates
            answered_correctly_u_count[row[0]] += len(previous_qu[row[0]])
            explanation_u_sum[row[0]] += int(row[4])*len(previous_qu[row[0]])
            elapsed_time_u_sum[row[0]] += row[3]*len(previous_qu[row[0]])
            if len(timestamp_u[row[0]]) == 4:
                timestamp_u[row[0]].pop(0)
            timestamp_u[row[0]].append(row[5])
            # ------------------------------------------------------------------
            #Client question features updates
            for (qid, q_diff) in zip(previous_qu[row[0]], container_uq_diff[row[0]]):
                if not np.isnan(q_diff):
                    diff_pu_sum[str(row[0]) + '_' + str(qs_parts[qid])] += q_diff
                    diff_pu_cnt[str(row[0]) + '_' + str(qs_parts[qid])] += 1
            container_uq_diff[row[0]] = [x for x in container_uq_diff[row[0]]
                                        if not np.isnan(x)]
            if len(container_uq_diff[row[0]]) > 0:
                last20_uq_diff[row[0]] += container_uq_diff[row[0]]
                diff_u_sum[row[0]] += sum(container_uq_diff[row[0]])
                diff_u_cnt[row[0]] += len(container_uq_diff[row[0]])
            if len(last20_uq_diff[row[0]]) > 20:
                for k in range(len(last20_uq_diff[row[0]]) - 20):
                    last20_uq_diff[row[0]].pop(0)
            # Question features updates
            for qid in previous_qu[row[0]]:
                answered_correctly_q_count[qid] += 1
                explanation_q_sum[qid] += int(row[4])
                elapsed_time_q_sum[qid] += row[3]
                # ------------------------------------------------------------------
                # Client Question updates
                answered_correctly_uq[str(row[0]) + '_' + str(qid)] += 1
                # ------------------------------------------------------------------
            if update:
                # ------------------------------------------------------------------
                # Client features updates
                answered_correctly_u_sum[row[0]] += sum(previous_qa[row[0]])
                if len(previous_qa[row[0]]) != 0:
                    last20_answers_u[row[0]] += previous_qa[row[0]]
                if len(last20_answers_u[row[0]]) > 20:
                    for k in range(len(last20_answers_u[row[0]]) - 20):
                        last20_answers_u[row[0]].pop(0)

                # ------------------------------------------------------------------
                # Question features updates
                for (qid, qa) in zip(previous_qu[row[0]], previous_qa[row[0]]):
                    answered_correctly_q_sum[qid] += qa
                    answered_correctly_pu_sum[str(row[0]) + '_' + str(qs_parts[qid])] += qa 
                    answered_correctly_pu_count[str(row[0]) + '_' + str(qs_parts[qid])] += 1
                # ------------------------------------------------------------------
                     
            
            
        # Client features assignation
        # ------------------------------------------------------------------
        if row_from_lecture[row[0]] != 0:
            row_from_lecture_u[num] = row_from_lecture[row[0]]
        else:
            row_from_lecture_u[num] = np.nan
            
        if answered_correctly_u_count[row[0]] != 0:
            answered_correctly_u_avg[num] = answered_correctly_u_sum[row[0]] / answered_correctly_u_count[row[0]]
            answered_u_cnt[num] = answered_correctly_u_count[row[0]]
            elapsed_time_u_avg[num] = elapsed_time_u_sum[row[0]] / answered_correctly_u_count[row[0]]
            explanation_u_avg[num] = explanation_u_sum[row[0]] / answered_correctly_u_count[row[0]]
        else:
            answered_correctly_u_avg[num] = np.nan
            elapsed_time_u_avg[num] = np.nan
            answered_u_cnt[num] = np.nan
            explanation_u_avg[num] = np.nan
            
        if answered_correctly_pu_count[str(row[0]) + '_' + str(row[8])] != 0:
            answered_correctly_pu_avg[num] = answered_correctly_pu_sum[str(row[0]) + '_' + str(row[8])] / answered_correctly_pu_count[str(row[0]) + '_' + str(row[8])]
        else:
            answered_correctly_pu_avg[num] = np.nan
            
        if len(last20_answers_u[row[0]]) != 0:
            answered_correctly_u_avg20[num] = np.mean(last20_answers_u[row[0]])
        else:
            answered_correctly_u_avg20[num] = np.nan
            
        if len(last20_uq_diff[row[0]]) > 0:
            difficulty_u_avg20[num] = np.mean(last20_uq_diff[row[0]])
        else:
            difficulty_u_avg20[num] = np.nan
            
        if diff_u_cnt[row[0]] != 0:
            difficulty_u_avg[num] = diff_u_sum[row[0]]/diff_u_cnt[row[0]]
        else:
            difficulty_u_avg[num] = np.nan
            
        if diff_pu_cnt[str(row[0]) + '_' + str(qs_parts[row[2]])] != 0:
            difficulty_pu_avg[num] = diff_pu_sum[str(row[0]) + '_' + str(qs_parts[row[2]])]/diff_pu_cnt[str(row[0]) + '_' + str(qs_parts[row[2]])]
        else:
            difficulty_pu_avg[num] = np.nan
            
        answered_pu_cnt[num] = answered_correctly_pu_count[str(row[0]) + '_' + str(row[8])]
                
        if len(timestamp_u[row[0]]) == 0:
            timestamp_u_recency_1[num] = np.nan
            timestamp_u_recency_2[num] = np.nan
            timestamp_u_recency_3[num] = np.nan
        elif timestamp_u[row[0]][-1] == row[5]:
            if len(timestamp_u[row[0]]) == 1:
                timestamp_u_recency_1[num] = np.nan
                timestamp_u_recency_2[num] = np.nan
                timestamp_u_recency_3[num] = np.nan
            elif len(timestamp_u[row[0]]) == 2:
                timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][0]
                timestamp_u_recency_2[num] = np.nan
                timestamp_u_recency_3[num] = np.nan
            elif len(timestamp_u[row[0]]) == 3:
                timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][1]
                timestamp_u_recency_2[num] = row[5] - timestamp_u[row[0]][0]
                timestamp_u_recency_3[num] = np.nan
            elif len(timestamp_u[row[0]]) == 4:
                timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][2]
                timestamp_u_recency_2[num] = row[5] - timestamp_u[row[0]][1]
                timestamp_u_recency_3[num] = row[5] - timestamp_u[row[0]][0]
        else:
            if len(timestamp_u[row[0]]) >= 3:
                timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][-1]
                timestamp_u_recency_2[num] = row[5] - timestamp_u[row[0]][-2]
                timestamp_u_recency_3[num] = row[5] - timestamp_u[row[0]][-3]
            elif len(timestamp_u[row[0]]) == 2:
                timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][-1]
                timestamp_u_recency_2[num] = row[5] - timestamp_u[row[0]][-2]
                timestamp_u_recency_3[num] = np.nan
            elif len(timestamp_u[row[0]]) == 1:
                timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][-1]
                timestamp_u_recency_2[num] = np.nan
                timestamp_u_recency_3[num] = np.nan
            

        if len(timestamp_u_incorrect[row[0]]) == 0:
            timestamp_u_incorrect_recency[num] = np.nan
        elif timestamp_u_incorrect[row[0]][-1] == row[5]:
            if len(timestamp_u_incorrect[row[0]]) == 1:
                timestamp_u_incorrect_recency[num] = np.nan
            else:
                timestamp_u_incorrect_recency[num] = row[5] - timestamp_u_incorrect[row[0]][0]
        else:
            timestamp_u_incorrect_recency[num] = row[5] - timestamp_u_incorrect[row[0]][-1]
            
        # ------------------------------------------------------------------
        # Question features assignation
        if answered_correctly_q_count[row[2]] != 0:
            answered_correctly_q_avg[num] = answered_correctly_q_sum[row[2]] / answered_correctly_q_count[row[2]]
            elapsed_time_q_avg[num] = elapsed_time_q_sum[row[2]] / answered_correctly_q_count[row[2]]
            explanation_q_avg[num] = explanation_q_sum[row[2]] / answered_correctly_q_count[row[2]]       
        else:
            answered_correctly_q_avg[num] = np.nan
            explanation_q_avg[num] = np.nan
            elapsed_time_q_avg[num] = np.nan
        # ------------------------------------------------------------------
        # Container features assignation
        if row[6] < last_container_u[row[0]] or \
        row[6] == last_wrong_container_u[row[0]]:
            last_wrong_container_u[row[0]] = row[6]
            wrong_container_u[num] = 1
        else:
            wrong_container_u[num] = 0
                
        # ------------------------------------------------------------------
        # Client Question assignation
        answered_correctly_uq_count[num] = answered_correctly_uq[str(row[0]) + '_' + str(row[2])]
        # ------------------------------------------------------------------
        # ------------------------------------------------------------------
        
        # Current row updates
        if last_container_u[row[0]] != row[6]:
            previous_qu[row[0]] = [row[2]]
            if answered_correctly_q_count[row[2]] != 0:
                container_uq_diff[row[0]] = [answered_correctly_q_sum[row[2]] / answered_correctly_q_count[row[2]]]
            else:
                container_uq_diff[row[0]]  = [np.nan]
            # Flag for training and inference
            if update:
                previous_qa[row[0]] = [row[1]]
                
        else:
            previous_qu[row[0]].append(row[2])
            if answered_correctly_q_count[row[2]] != 0:
                container_uq_diff[row[0]].append(
                    answered_correctly_q_sum[row[2]] / answered_correctly_q_count[row[2]])
            else:
                container_uq_diff[row[0]].append(np.nan)
            if update:
                previous_qa[row[0]].append(row[1])
        if update:
            if row[1] == 0:
                if len(timestamp_u_incorrect[row[0]]) == 0 \
                or row[5] != timestamp_u_incorrect[row[0]][-1]:
                    if len(timestamp_u_incorrect[row[0]]) == 2:
                        timestamp_u_incorrect[row[0]].pop(0)
                    timestamp_u_incorrect[row[0]].append(row[5])


        last_container_u[row[0]] = row[6]
            
    user_df = pd.DataFrame({'answered_correctly_u_avg': answered_correctly_u_avg, 'elapsed_time_u_avg': elapsed_time_u_avg, 'explanation_u_avg': explanation_u_avg, 
                            'answered_correctly_q_avg': answered_correctly_q_avg, 'elapsed_time_q_avg': elapsed_time_q_avg, 'explanation_q_avg': explanation_q_avg, 
                            'answered_correctly_uq_count': answered_correctly_uq_count, 'timestamp_u_recency_1': timestamp_u_recency_1, 'timestamp_u_recency_2': timestamp_u_recency_2,
                            'timestamp_u_recency_3': timestamp_u_recency_3, 'timestamp_u_incorrect_recency': timestamp_u_incorrect_recency,
                            'wrong_container_u': wrong_container_u,
                            'answered_u_cnt': answered_u_cnt,
                            'answered_correctly_pu_avg': answered_correctly_pu_avg,
                            'answered_pu_cnt': answered_pu_cnt,
                            'answered_correctly_u_avg20': answered_correctly_u_avg20,
                            'difficulty_u_avg20': difficulty_u_avg20,
                            'difficulty_u_avg': difficulty_u_avg,
                            'difficulty_pu_avg': difficulty_pu_avg,
                            'row_from_lecture_u': row_from_lecture_u})
    
    df = pd.concat([df.loc[df.answered_correctly != -1].reset_index(drop=True),
                    user_df], axis = 1)
    return df
        
def update_features(df, answered_correctly_u_sum, answered_correctly_q_sum, timestamp_u_incorrect, last_container_u, previous_qa, answered_correctly_pu_count, answered_correctly_pu_sum):
    for row in df[['user_id', 'answered_correctly', 'content_id', 'content_type_id', 'timestamp', 'task_container_id']].values:
        if row[3] == 0:
            # ------------------------------------------------------------------
            # Client features updates
            answered_correctly_u_sum[row[0]] += row[1]
            if row[1] == 0:
                if len(timestamp_u_incorrect[row[0]]) == 2:
                    timestamp_u_incorrect[row[0]].pop(0)
                timestamp_u_incorrect[row[0]].append(row[4])
            # ------------------------------------------------------------------
            # Question features updates
            answered_correctly_q_sum[row[2]] += row[1]
            answered_correctly_pu_sum[str(row[0]) + '_' + str(qs_parts[row[2]])] += row[1] 
            answered_correctly_pu_count[str(row[0]) + '_' + str(qs_parts[row[2]])] += 1
            # ------------------------------------------------------------------
            
    return

def read_and_preprocess(feature_engineering = False):
    
    train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
    valid_pickle = '../input/riiid-cross-validation-files/cv1_valid.pickle'
    question_file = '../input/riiid-test-answer-prediction/questions.csv'
    lecture_file = '../input/riiid-test-answer-prediction/lectures.csv'
    
    # Read data
    feld_needed = ['timestamp', 'user_id', 'answered_correctly', 'content_id', 'content_type_id', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'task_container_id']
    train = pd.read_pickle(train_pickle)[feld_needed]
    valid = pd.read_pickle(valid_pickle)[feld_needed]
    # Delete some trianing data to don't have ram problems
    if feature_engineering:
        train = train.iloc[-40000000:]
    
    # Filter by content_type_id to discard lectures
    train['row_n'] = 1
    train['row_n'] = train['row_n'].cumsum()
    valid['row_n'] = 1
    valid['row_n'] = valid['row_n'].cumsum()
    train_lectures = train.loc[train.content_type_id == True]
    train = train.loc[train.content_type_id == False]#.reset_index(drop = True)
    valid_lectures = valid.loc[valid.content_type_id == True]
    valid = valid.loc[valid.content_type_id == False]#.reset_index(drop = True)
    
    # Merge with question dataframe
    questions_df = pd.read_csv(question_file)
    questions_df['part'] = questions_df['part'].astype(np.int32)
    questions_df['bundle_id'] = questions_df['bundle_id'].astype(np.int32)
    questions_df['tag'] = questions_df['tags'].apply(lambda ts: [int(x) for x in str(ts).split() if x != 'nan'])
    lectures_df = pd.read_csv(lecture_file)
    lectures_df['part'] = lectures_df['part'].astype(np.int32)
    #
    
    train = pd.merge(train, questions_df[['question_id', 'part', 'tag']], left_on = 'content_id', right_on = 'question_id', how = 'left')
    len_train = len(train)
    train_lectures = pd.merge(train_lectures, lectures_df[['lecture_id', 'part', 'tag']], left_on = 'content_id', right_on = 'lecture_id', how = 'left')
    train = pd.concat([train, train_lectures]).sort_values(['row_n'])
    #train = train.sort_values(['row_n'])
    train.drop(['question_id', 'lecture_id', 'row_n'], axis=1, inplace=True)
    valid = pd.merge(valid, questions_df[['question_id', 'part', 'tag']], left_on = 'content_id', right_on = 'question_id', how = 'left')
    len_valid = len(valid)
    valid_lectures = pd.merge(valid_lectures, lectures_df[['lecture_id', 'part', 'tag']], left_on = 'content_id', right_on = 'lecture_id', how = 'left')
    valid = pd.concat([valid, valid_lectures]).sort_values(['row_n'])
    valid.drop(['question_id', 'lecture_id', 'row_n'], axis=1, inplace=True)
    #valid = valid.sort_values(['row_n'])
    
    # Changing dtype to avoid lightgbm error
    train['prior_question_had_explanation'] = train.prior_question_had_explanation.fillna(False).astype('int8')
    valid['prior_question_had_explanation'] = valid.prior_question_had_explanation.fillna(False).astype('int8')
    
    # Fill prior question elapsed time with the mean
    prior_question_elapsed_time_mean = train['prior_question_elapsed_time'].dropna().mean()
    train['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace = True)
    valid['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace = True)
    
    
    
    # Client dictionaries
    answered_correctly_u_count = defaultdict(int)
    answered_correctly_u_sum = defaultdict(int)
    answered_correctly_pu_count = defaultdict(int)
    answered_correctly_pu_sum = defaultdict(int)
    elapsed_time_u_sum = defaultdict(int)
    explanation_u_sum = defaultdict(int)
    timestamp_u = defaultdict(list)
    timestamp_u_incorrect = defaultdict(list)
    last20_answers_u = defaultdict(list)
    diff_u_sum = defaultdict(float) 
    diff_u_cnt = defaultdict(int)
    diff_pu_sum = defaultdict(float)
    diff_pu_cnt = defaultdict(int)
    row_from_lecture = defaultdict(int)
                                                        
    
    # Question dictionaries
    answered_correctly_q_count = defaultdict(int)
    answered_correctly_q_sum = defaultdict(int)
    previous_qu = defaultdict(list)
    elapsed_time_q_sum = defaultdict(int)
    explanation_q_sum = defaultdict(int)
    previous_qa = defaultdict(list)
    
    #Container dicts
    last_container_u = defaultdict(lambda: -1)
    last_wrong_container_u = defaultdict(lambda: -1)
    
    # Client Question dictionary
    answered_correctly_uq = defaultdict(int)
    last20_uq_diff = defaultdict(list)
    container_uq_diff = defaultdict(list)
    
    print('User feature calculation started...')
    print('\n')
    train = add_features(train, len_train, answered_correctly_u_count, answered_correctly_u_sum, elapsed_time_u_sum, explanation_u_sum, timestamp_u, timestamp_u_incorrect, answered_correctly_q_count, answered_correctly_q_sum, elapsed_time_q_sum, explanation_q_sum, answered_correctly_uq, last_wrong_container_u, last_container_u, previous_qu, previous_qa, answered_correctly_pu_count, answered_correctly_pu_sum, last20_answers_u, last20_uq_diff, container_uq_diff, diff_u_sum, diff_u_cnt, diff_pu_sum, diff_pu_cnt, row_from_lecture)
    valid = add_features(valid, len_valid, answered_correctly_u_count, answered_correctly_u_sum, elapsed_time_u_sum, explanation_u_sum, timestamp_u, timestamp_u_incorrect, answered_correctly_q_count, answered_correctly_q_sum, elapsed_time_q_sum, explanation_q_sum, answered_correctly_uq, last_wrong_container_u, last_container_u, previous_qu, previous_qa, answered_correctly_pu_count, answered_correctly_pu_sum, last20_answers_u, last20_uq_diff, container_uq_diff, diff_u_sum, diff_u_cnt, diff_pu_sum, diff_pu_cnt, row_from_lecture)
    gc.collect()
    print('User feature calculation completed...')
    print('\n')
    
    features_dicts = {
        'answered_correctly_u_count': answered_correctly_u_count,
        'answered_correctly_u_sum': answered_correctly_u_sum,
        'elapsed_time_u_sum': elapsed_time_u_sum,
        'explanation_u_sum': explanation_u_sum,
        'answered_correctly_q_count': answered_correctly_q_count,
        'answered_correctly_q_sum': answered_correctly_q_sum,
        'elapsed_time_q_sum': elapsed_time_q_sum,
        'explanation_q_sum': explanation_q_sum,
        'answered_correctly_uq': answered_correctly_uq,
        'timestamp_u': timestamp_u,
        'timestamp_u_incorrect': timestamp_u_incorrect,
        'last_wrong_container_u': last_wrong_container_u,
        'last_container_u': last_container_u,
        'previous_qu': previous_qu,
        'previous_qa': previous_qa,
        'answered_correctly_pu_count': answered_correctly_pu_count,
        'answered_correctly_pu_sum': answered_correctly_pu_sum,
        'last20_answers_u': last20_answers_u,
        'last20_uq_diff': last20_uq_diff,
        'container_uq_diff': container_uq_diff,
        'diff_u_sum': diff_u_sum,
        'diff_u_cnt': diff_u_cnt,
        'diff_pu_sum': diff_pu_sum,
        'diff_pu_cnt': diff_pu_cnt,
        'row_from_lecture': row_from_lecture
    }
    
    return train, valid, questions_df, lectures_df, prior_question_elapsed_time_mean, features_dicts

# Function for training and evaluation
def train_and_evaluate(train, valid, feature_engineering = False):
    
    TARGET = 'answered_correctly'
    # Features to train and predict
    FEATURES = ['prior_question_elapsed_time', 'prior_question_had_explanation',
                'part', 'answered_correctly_u_avg', 'elapsed_time_u_avg', 'explanation_u_avg',
                'answered_correctly_q_avg', 'elapsed_time_q_avg', 'explanation_q_avg', 'answered_correctly_uq_count', 'timestamp_u_recency_1', 'timestamp_u_recency_2', 'timestamp_u_recency_3', 
                'timestamp_u_incorrect_recency',
               'wrong_container_u', 'answered_u_cnt',
               'answered_correctly_pu_avg',
               'answered_pu_cnt', 'answered_correctly_u_avg20',
                'difficulty_u_avg20', 'difficulty_u_avg',
                'difficulty_pu_avg', 'row_from_lecture_u'
               ]
    
    # Delete some training data to experiment faster
    if feature_engineering:
        train = train.sample(15000000, random_state = SEED)
    gc.collect()
    print(f'Traning with {train.shape[0]} rows and {len(FEATURES)} features')    
    drop_cols = list(set(train.columns) - set(FEATURES))
    y_train = train[TARGET]
    y_val = valid[TARGET]
    # Drop unnecessary columns
    train.drop(drop_cols, axis = 1, inplace = True)
    valid.drop(drop_cols, axis = 1, inplace = True)
    gc.collect()
    
    lgb_train = lgb.Dataset(train[FEATURES], y_train)
    lgb_valid = lgb.Dataset(valid[FEATURES], y_val)
    del train, y_train
    gc.collect()
    
    params = {'objective': 'binary', 
              'seed': SEED,
              'metric': 'auc',
              'num_leaves': 500,
              'feature_fraction': 0.8,
              'bagging_freq': 10,
              'bagging_fraction': 0.80
             }
    
    model = lgb.train(
        params = params,
        train_set = lgb_train,
        num_boost_round = 10000,
        valid_sets = [lgb_train, lgb_valid],
        early_stopping_rounds = 10,
        verbose_eval = 50
    )
    
    print('Our Roc Auc score for the validation data is:', roc_auc_score(y_val, model.predict(valid[FEATURES])))
    
    feature_importance = model.feature_importance()
    feature_importance = pd.DataFrame({'Features': FEATURES, 'Importance': feature_importance}).sort_values('Importance', ascending = False)
    
    fig = plt.figure(figsize = (10, 10))
    fig.suptitle('Feature Importance', fontsize = 20)
    plt.tick_params(axis = 'x', labelsize = 12)
    plt.tick_params(axis = 'y', labelsize = 12)
    plt.xlabel('Importance', fontsize = 15)
    plt.ylabel('Features', fontsize = 15)
    sns.barplot(x = feature_importance['Importance'], y = feature_importance['Features'], orient = 'h')
    plt.show()
    
    return TARGET, FEATURES, model

    
train, valid, questions_df, lectures_df, prior_question_elapsed_time_mean, features_dicts = read_and_preprocess(feature_engineering = True)
#TARGET, FEATURES, model = train_and_evaluate(train, valid, feature_engineering = True)
#inference(TARGET, FEATURES, model, questions_df, lectures_df, prior_question_elapsed_time_mean, features_dicts)

In [ ]:
prior_question_elapsed_time_mean

In [ ]:
import gc
train.to_pickle('train3.pkl')
valid.to_pickle('valid3.pkl')
#del train
#del valid
#train = pd.read_pickle('train3.pkl')
#valid = pd.read_pickle('valid3.pkl')

In [ ]:
TARGET, FEATURES, model = train_and_evaluate(train, valid, feature_engineering = True)




In [ ]:
#defaultdict to dict, drop nested dicts
def default_to_dict(features_dict):
    res = {}
    for k1 in features_dict.keys():
        #res[k1] = {}
        if k1 == 'answered_correctly_uq':
            continue
        for k2 in features_dict[k1].keys():
            if type(features_dict[k1][k2]) == type(defaultdict()):
                break#res[k1][k2] = dict(features_dict[k1][k2])
            else:
                res[k1] = dict(features_dict[k1])
                break
            
    return res

In [ ]:
#save dict to file
import pickle
res_dict = default_to_dict(features_dicts)
with open('features_f2l.pickle', 'wb') as handle:
    pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)





In [ ]:
import joblib
# save model
joblib.dump(model, 'lgbf2.pkl')

In [ ]:
#save nested dict to file
res = pd.DataFrame({'uc_id': sorted(list(features_dicts['answered_correctly_uq'].keys())),
                    'count': [features_dicts['answered_correctly_uq'][k]
                        for k in sorted(
                            list(features_dicts['answered_correctly_uq'].keys()))]})

In [ ]:
res.to_csv('uq.csv')